In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
import os
from python_speech_features import mfcc
from python_speech_features import delta
from python_speech_features import logfbank
import scipy
from scipy.signal import hamming
from scipy.fftpack import dct
from numpy.fft import fft
from hmmlearn import hmm
from tensorflow.keras import Sequential, optimizers
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, Activation, Flatten, Dense, LSTM

# READ ME

Welcome to my speech recognition data project! In this project I take in a bunch of samples of people saying the words "no", "yes", "one", "two", "tree", "bed", "bird", and a few other words. There are about 3000 samples per each category.

In my attempt to complete this project I made a few different classes like "process_speech_data", "HMM_trainer", and "speech_recognize" to name a few. I also tried to used a Hidden Markov Model and I tried to use LSTM neural networks.

If you are simply interested in the method that worked, please scroll down to the section near the bottom titled "Final LSTM Algorithm". I kept a lot of the code from my failed attempts to demonstrate the work that I put into this project. In addition to that, I do end up using the classes that pre-process my data. I tried to create classes that act as a wrapper for my program.

Enjoy!

James Longstaff - jklongstaff@gmail.com

In [1]:
class process_speech_data:
    def __init__(self, path, filenames, samplerate = 16000):
        self.path = path
        self.filenames = filenames
        self.whole_wavs = list()
        self.framed_wavs = list()
        self.samplerate = samplerate
        self.mfcc_list = list()
    def load_waves(self, ms_frame_len = 0.025):
        """
        This function loads the filenames of a function and the path and then outputs a tuple of two numpy arrays
        that are a list of the wav formats represented as numbers.
        """
        
        all_waves = []
        waves_framed = []
        frames = np.empty((400,))
        num_frames_per_wav = int(self.samplerate*ms_frame_len)
        times_too_short = 0
        
        for filename in self.filenames:
            wav = wavfile.read(self.path + '/' + filename)
            
            if len(wav[1]) < self.samplerate:

                d = np.pad(wav[1],(self.samplerate - len(wav[1]), 0), mode = 'constant')

            elif len(wav[1]) > self.samplerate:
                raise ValueError("There are more than {} samples!".format(str(self.samplerate)))

            else:
                d = wav[1][0:self.samplerate]

            
            
            spaced400_to16000 = np.arange(800,self.samplerate + 400,400)


            frames = d[0:num_frames_per_wav]
            old_j = num_frames_per_wav
            
            for j in spaced400_to16000:
                frames = np.vstack((frames,d[old_j:j]))

                old_j = j


            d_with_frames = (d,frames)   
            all_waves.append(d_with_frames)
        
        self.whole_wavs = [wav[0] for wav in all_waves]
        self.framed_wavs = [wav[1] for wav in all_waves]
        return all_waves

    def get_mfcc(self):
        whole_wavs = self.whole_wavs
        mfcc_list = []
        for sig in whole_wavs:
            mfcc_feat = mfcc(sig,self.samplerate)
            d_mfcc_feat = delta(mfcc_feat, 2)
            fbank_feat = logfbank(sig,self.samplerate)
            mfcc_list.append(fbank_feat.T)
        self.mfcc_list = mfcc_list
  

In [ ]:
class HMM_trainer:
    def __init__(self,model_name = "GaussianHMM", num_states = 4, cov_type = 'diag', n_iter = 1000):
        self.model_name = model_name
        self.num_states = num_states
        self.cov_type = cov_type
        self.n_iter = n_iter
        self.models = []

        
        if self.model_name == "GaussianHMM":
            self.model = hmm.GaussianHMM(n_components = self.num_states, covariance_type = self.cov_type,n_iter = self.n_iter)
        
        else:
            raise TypeError("Invalid model type")
            
    def train(self,X, lengths):
        np.seterr(all='ignore')
        self.models.append(self.model.fit(X,lengths))
        
    def get_score(self,input_data , is_model_extra = False, extra_model = None):
        if is_model_extra == True:
            return extra_model.score(input_data)
        else:
            return self.model.score(input_data)
    def save(self,i):
        model = self.model
        joblib.dump(model, self.model_name  + str(i+1) + ".pkl")
   
     


In [3]:

class speech_recognize:
    def __init__(self, mfcc_lists, input_files, classes, samplerate = 16000):
        self.mfcc_lists = mfcc_lists
        self.hmm_models = []
        self.input_files = input_files
        self.classes = classes
        self.samplerate = samplerate
    def get_hmm_models(self):
        
        def concat(mfcc_list):
            X = np.ones(6)
            lengths = []
            for i in range(len(mfcc_list)):
                array = mfcc_list[i]
                if i > 0:
                    X = np.concatenate([X,array], axis = 1)
                    lengths.append(array.shape[1])
                else:
                    print(i)
                    
                    X = array
                    print(array.shape)
                    print(array.shape[1])
                    lengths.append(array.shape[1])
                    print("Lengths: ", lengths,"\nX shape: ")
            print(X.shape)
            return X, lengths
            
        j = 0
        for mfcc_list in mfcc_lists:
            X, lengths = concat(mfcc_list)
            hmm_trainer = HMM_trainer(num_states = 8)
            hmm_trainer.train(X.T,lengths)
            print("Done training the HMM model")
            hmm_trainer.save(j)
            print("Done saving the model")
            self.hmm_models.append((hmm_trainer,j))
            print(len(self.hmm_models))
            j += 1
            hmm_trainer = None
            
            
    def score_hmm_models(self):
        
        for input_file in self.input_files:
            
            sampling_freq, sig = wavfile.read(input_file)

            if len(sig) < self.samplerate:

                sig = np.pad(sig,(self.samplerate - len(sig), 0), mode = 'constant')

            elif len(sig) > self.samplerate:
                raise ValueError("There are more than {} samples!".format(str(self.samplerate)))

            else:
                sig = sig[0:self.samplerate]

            
            mfcc_feat = mfcc(sig,sampling_freq)
            #print("mfcc_feat shape: ", mfcc_feat.shape)
            d_mfcc_feat = delta(mfcc_feat, 2)
            #print("d_mfcc_feat shape: ", d_mfcc_feat.shape)
            fbank_feat = logfbank(sig,sampling_freq)
            #print("fbank_feat shape: ", fbank_feat.shape)
            
            max_score = [float("-inf")]
            output_label = [float("-inf")]
            
            for item in self.hmm_models:
                hmm_model, label = item
                print("This is the interger corresponding to the label: ",label)
                score = hmm_model.get_score(fbank_feat)
                if score > max_score:
                    max_score = score
                    output_label = label
            print("The input file ", input_file, " says ", self.classes[output_label])
            print("\n\n")
            

In [ ]:
yes_path = "C:/Users/kaweie/Documents/speech/yes"
no_path = "C:/Users/kaweie/Documents/speech/no"

bird_path = "C:/Users/kaweie/Documents/speech/bird"
dog_path = "C:/Users/kaweie/Documents/speech/dog"
cat_path = "C:/Users/kaweie/Documents/speech/cat"
bed_path = "C:/Users/kaweie/Documents/speech/bed"
wow_path = "C:/Users/kaweie/Documents/speech/wow"
stop_path = "C:/Users/kaweie/Documents/speech/stop"

one_path = "C:/Users/kaweie/Documents/speech/one"
two_path = "C:/Users/kaweie/Documents/speech/two"
three_path = "C:/Users/kaweie/Documents/speech/three"
four_path = "C:/Users/kaweie/Documents/speech/four"
five_path = "C:/Users/kaweie/Documents/speech/five"
six_path = "C:/Users/kaweie/Documents/speech/six"



yes_filenames = os.listdir(yes_path)
no_filenames = os.listdir(no_path)


one_filenames = os.listdir(one_path)
two_filenames = os.listdir(two_path)
three_filenames = os.listdir(three_path)
four_filenames = os.listdir(four_path)
five_filenames = os.listdir(five_path)
six_filenames = os.listdir(six_path)

yes = process_speech_data(yes_path, yes_filenames)
wavs_yes = yes.load_waves()
yes.get_mfcc()

no =  process_speech_data(no_path, no_filenames)
wavs_no = no.load_waves()
no.get_mfcc()

one = process_speech_data(one_path, one_filenames)
wavs_one = one.load_waves()
one.get_mfcc()

two =  process_speech_data(two_path, two_filenames)
wavs_two = two.load_waves()
two.get_mfcc()

three = process_speech_data(three_path, three_filenames)
wavs_three = three.load_waves()
three.get_mfcc()

four =  process_speech_data(four_path, four_filenames)
wavs_four = four.load_waves()
four.get_mfcc()

five = process_speech_data(five_path, five_filenames)
wavs_five = five.load_waves()
five.get_mfcc()

six =  process_speech_data(six_path, six_filenames)
wavs_six = six.load_waves()
six.get_mfcc()

on =  process_speech_data(on_path, on_filenames)
wavs_on = on.load_waves()
on.get_mfcc()

off =  process_speech_data(off_path, off_filenames)
wavs_off = off.load_waves()
off.get_mfcc()



mfcc_list_yes = yes.mfcc_list
mfcc_list_no = no.mfcc_list

mfcc_list_one = one.mfcc_list
mfcc_list_two = two.mfcc_list
mfcc_list_three = three.mfcc_list
mfcc_list_four = four.mfcc_list
mfcc_list_five = five.mfcc_list
mfcc_list_six = six.mfcc_list




In [ ]:
cat_filenames = os.listdir(cat_path)
cat =  process_speech_data(cat_path, cat_filenames)
wavs_cat = cat.load_waves()
cat.get_mfcc()

mfcc_list_cat = cat.mfcc_list

speech = speech_recognize([mfcc_list_cat],test_inputs,["cat"])

print("Now Training HMM Models")
speech.get_hmm_models()
print("Finished training HMM models\n\n")

In [ ]:
mfcc_lists = [mfcc_list_yes, mfcc_list_no]#,mfcc_list_one, mfcc_list_two,
             # mfcc_list_three,mfcc_list_four,mfcc_list_five, mfcc_list_six]
classes = ["yes","no"]#, "one", "two", "three", "four", "five", "six"]

test_inputs = ['C:/Users/kaweie/Documents/speech/test/one1.wav',
               'C:/Users/kaweie/Documents/speech/test/one2.wav',
               'C:/Users/kaweie/Documents/speech/test/one3.wav',
               'C:/Users/kaweie/Documents/speech/test/one4.wav',
               'C:/Users/kaweie/Documents/speech/test/two1.wav',
               'C:/Users/kaweie/Documents/speech/test/two2.wav',
               'C:/Users/kaweie/Documents/speech/test/two3.wav',
               'C:/Users/kaweie/Documents/speech/test/two4.wav',
               'C:/Users/kaweie/Documents/speech/test/three1.wav',
               'C:/Users/kaweie/Documents/speech/test/three2.wav',
               'C:/Users/kaweie/Documents/speech/test/three3.wav',
               'C:/Users/kaweie/Documents/speech/test/three4.wav',
               'C:/Users/kaweie/Documents/speech/test/four1.wav',
               'C:/Users/kaweie/Documents/speech/test/four2.wav',
               'C:/Users/kaweie/Documents/speech/test/four3.wav',
               'C:/Users/kaweie/Documents/speech/test/four4.wav',
               'C:/Users/kaweie/Documents/speech/test/five1.wav',
               'C:/Users/kaweie/Documents/speech/test/five2.wav',
               'C:/Users/kaweie/Documents/speech/test/five3.wav',
               'C:/Users/kaweie/Documents/speech/test/five4.wav',
               'C:/Users/kaweie/Documents/speech/test/six1.wav',
               'C:/Users/kaweie/Documents/speech/test/six2.wav',
               'C:/Users/kaweie/Documents/speech/test/six3.wav',
               'C:/Users/kaweie/Documents/speech/test/six4.wav',
               'C:/Users/kaweie/Documents/speech/test/no1.wav',
               'C:/Users/kaweie/Documents/speech/test/no2.wav',
               'C:/Users/kaweie/Documents/speech/test/no3.wav',
               'C:/Users/kaweie/Documents/speech/test/no4.wav',
               'C:/Users/kaweie/Documents/speech/test/yes1.wav',
               'C:/Users/kaweie/Documents/speech/test/yes2.wav',
               'C:/Users/kaweie/Documents/speech/test/yes3.wav',
               'C:/Users/kaweie/Documents/speech/test/yes4.wav']

if yes.samplerate != no.samplerate != on.samplerate != off.samplerate:
    raise ValueError("The wav files in each class do not have the same sample rate.") 
speech = speech_recognize(mfcc_lists,test_inputs,classes)
print("Now Training HMM Models")
speech.get_hmm_models()
print("Finished training HMM models\n\n")
print("Now scoring the models against the test_inputs.\n\n")
#speech.score_hmm_models()

# Recording my own sound

In this next code cell I attempt to record my own sounds. Since I was not familiar with how this worked, I used someone else's code to record sound. Although it could record quite well, the produced samples did not work well in the classifier. Maybe I need a better microphone or I need to think of a better way to concisely record the audio?

In [ ]:
from sys import byteorder
from array import array
from struct import pack

import pyaudio
import wave

THRESHOLD = 500
CHUNK_SIZE = 1024
FORMAT = pyaudio.paInt16
RATE = 44100

def is_silent(snd_data):
    "Returns 'True' if below the 'silent' threshold"
    return max(snd_data) < THRESHOLD

def normalize(snd_data):
    "Average the volume out"
    MAXIMUM = 16384
    times = float(MAXIMUM)/max(abs(i) for i in snd_data)

    r = array('h')
    for i in snd_data:
        r.append(int(i*times))
    return r

def trim(snd_data):
    "Trim the blank spots at the start and end"
    def _trim(snd_data):
        snd_started = False
        r = array('h')

        for i in snd_data:
            if not snd_started and abs(i)>THRESHOLD:
                snd_started = True
                r.append(i)

            elif snd_started:
                r.append(i)
        return r

    # Trim to the left
    snd_data = _trim(snd_data)

    # Trim to the right
    snd_data.reverse()
    snd_data = _trim(snd_data)
    snd_data.reverse()
    return snd_data

def add_silence(snd_data, seconds):
    "Add silence to the start and end of 'snd_data' of length 'seconds' (float)"
    r = array('h', [0 for i in range(int(seconds*RATE))])
    r.extend(snd_data)
    r.extend([0 for i in range(int(seconds*RATE))])
    return r


def record():
    """
    Record a word or words from the microphone and 
    return the data as an array of signed shorts.

    Normalizes the audio, trims silence from the 
    start and end, and pads with 0.5 seconds of 
    blank sound to make sure VLC et al can play 
    it without getting chopped off.
    """
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT, channels=1, rate=RATE,
        input=True, output=True,
        frames_per_buffer=CHUNK_SIZE)

    num_silent = 0
    snd_started = False

    r = array('h')

    while 1:
        # little endian, signed short
        snd_data = array('h', stream.read(CHUNK_SIZE))
        if byteorder == 'big':
            snd_data.byteswap()
        r.extend(snd_data)

        silent = is_silent(snd_data)

        if silent and snd_started:
            num_silent += 1
        elif not silent and not snd_started:
            snd_started = True

        if snd_started and num_silent > 30:
            break

    sample_width = p.get_sample_size(FORMAT)
    stream.stop_stream()
    stream.close()
    p.terminate()

    r = normalize(r)
    r = trim(r)
    r = add_silence(r, 0.5)
    return sample_width, r

def record_to_file(path):
    "Records from the microphone and outputs the resulting data to 'path'"
    sample_width, data = record()
    data = pack('<' + ('h'*len(data)), *data)

    wf = wave.open(path, 'wb')
    wf.setnchannels(1)
    wf.setsampwidth(sample_width)
    wf.setframerate(RATE)
    wf.writeframes(data)
    wf.close()

if __name__ == '__main__':
    print("please speak a word into the microphone")
    record_to_file('demo.wav')
    print("done - result written to demo.wav")
    

In [ ]:
print("GaussianHMM" + str(2) +".pkl")
models = [joblib.load("GaussianHMM" + str(i+1) +".pkl") for i in range(2)]
models.append(joblib.load("GaussianHMM9.pkl"))
classes = ["yes","no","cat"]#, "one", "two", "three", "four", "five", "six"]
samplerate = 16000

input_file = 'C:/Users/kaweie/Documents/speech/test/yes4.wav'

sampling_freq, sig = wavfile.read(input_file)


mfcc_feat = mfcc(sig,sampling_freq,nfft = 1200)
d_mfcc_feat = delta(mfcc_feat, 2)
fbank_feat = logfbank(sig,sampling_freq,nfft = 1200)


max_score = [float("-inf")]
output_label = [float("-inf")]

for i in range(len(models)):
    label = i
    hmm_trainer = HMM_trainer()
    score = hmm_trainer.get_score(fbank_feat, is_model_extra = True, extra_model = models[i])
    print(score)
    if score > max_score:
        max_score = score
        output_label = label
print("The input file ", input_file, " says ", classes[output_label])
print("\n\n")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)
print(np.array(X_train).shape)
print(np.array(X_test).shape)

X_train = np.array(X_train)
X_test = np.array(X_test)
X_train = X_train.reshape((X_train.shape[0],X_train.shape[1],X_train.shape[2],1))
X_test = X_test.reshape((X_test.shape[0],X_test.shape[1],X_test.shape[2],1))

print(y_train[3214])
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
print(y_train[3214])
print(y_train.shape)

print(X_train.shape)
print(X_test.shape)
batch_size = 32


In [ ]:
model = Sequential()

model.add(Conv2D(filters = 128,kernel_size=(3,3), padding = 'VALID',input_shape = (26,99,1)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(rate = 0.5))

model.add(Conv2D(filters = 128,kernel_size=(3,3), padding='SAME'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(rate = 0.5))

model.add(Conv2D(filters = 128,kernel_size=(3,3), padding = 'SAME'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(rate = 0.5))

#model.add(Conv2D(filters = 128,kernel_size=(5,5)))
#model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size = (2,2)))
#model.add(Dropout(rate = 0.5))

model.add(Flatten())

model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(4))
model.add(Activation('sigmoid'))

model.compile(optimizer = optimizers.Adam(lr=0.001), 
              loss = "categorical_crossentropy",
              metrics=['accuracy'])
model.fit(X_train, y_train, batch_size = batch_size, epochs = 3, verbose = 1, validation_data =(X_test,y_test))

In [ ]:

plt.title('Filter bank')

mfcc_list = r.mfcc_list
print(mfcc_list[0])
plt.imshow(mfcc_list[0])
plt.show()

plt.title('Filter bank')
plt.imshow(mfcc_list[1])
plt.show()

plt.title('Filter bank')
plt.imshow(mfcc_list[2])
plt.show()

plt.title('Filter bank')
plt.imshow(mfcc_list[3])
plt.show()

plt.title('Filter bank')
plt.imshow(mfcc_list[4])
plt.show()

plt.title('Filter bank')
plt.imshow(mfcc_list[5])
plt.show()

In [ ]:
no_path = "C:/Users/kaweie/Documents/speech/no"
no_filenames = os.listdir(no_path)
print(len(no_filenames))
d = speech_recognize(no_path, no_filenames)
wavs_no = d.load_waves()
d.get_mfcc()
mfcc_list = d.mfcc_list
print(mfcc_list[0].shape)
plt.imshow(mfcc_list[0].T)
plt.show()

plt.title('Filter bank')
plt.imshow(mfcc_list[1].T)
plt.show()

plt.title('Filter bank')
plt.imshow(mfcc_list[2].T)
plt.show()

plt.title('Filter bank')
plt.imshow(mfcc_list[3].T)
plt.show()

plt.title('Filter bank')
plt.imshow(mfcc_list[4].T)
plt.show()

plt.title('Filter bank')
plt.imshow(mfcc_list[5].T)
plt.show()

plt.title('Filter bank')
plt.imshow(mfcc_list[6].T)
plt.show()

plt.title('Filter bank')
plt.imshow(mfcc_list[7].T)
plt.show()

plt.title('Filter bank')
plt.imshow(mfcc_list[8].T)
plt.show()

plt.title('Filter bank')
plt.imshow(mfcc_list[9].T)
plt.show()

plt.title('Filter bank')
plt.imshow(mfcc_list[10].T)
plt.show()

plt.title('Filter bank')
plt.imshow(mfcc_list[11].T)
plt.show()

In [ ]:
Mel = lambda f: 1125.*np.log(1 + f/700.)
Mel_inv = lambda m: 700.*(np.exp(m/1125.) - 1)

def mel_filterbank(min_freq = 300, max_freq = 16000):
    Hertz_freqs = np.linspace(min_freq, max_freq)
    mel_filters = Mel(Hertz_freqs)
    

In [ ]:
frames = wavs[:][1]

for frame in frames:
    N = len(frame)
    hamm_window = scipy.signal.hamming(N)
    
    DFT = [frame*hamm_window*np.exp(-1j*2*np.pi*k*n/N) for n in range(N)]
    Peridogram = 1./N * np.abs(DFT)**2

In [ ]:
part = wavs_yes[0][1]
print(part)
print(type(part))
max_freq = np.abs(part[np.argmax(part)])
print("Index of max value:")
print(np.argmax(part))
print(max_freq)
min_freq = np.abs(part[np.argmin(part)])
print(min_freq)

new_part = scipy.int16(part/np.linalg.norm(part)*32767)
print("Index of max value:")
print(np.argmax(new_part))
print(new_part[np.argmax(new_part)])

plt.plot(scipy.int16(part))
plt.show()
plt.plot(new_part)
plt.show()

In [ ]:
def get_feat(wavs):
    for wav in wavs:
        rate = wav[0]
        sig = wav[1]
        mfcc_feat = mfcc(sig,rate)
        d_mfcc_feat = delta(mfcc_feat,2)
        fbank_feat = logfbank(sig,rate)
        print("This is the rate:")
        print(rate)
        
        print("\nThis is the signal:")
        print(sig)
        
        print("\nThis is the mfcc_feat shape")
        print(mfcc_feat.shape)
        
        print("\nThis is the mfcc_feat")
        print(mfcc_feat)
        
        print("\nThis is the d_mfcc_feat shape")
        print(d_mfcc_feat.shape)
        
        print("\nThis is the dmfccfeat")
        print(d_mfcc_feat)
        
        print("\nThis is the fbank_feat")
        print(fbank_feat[1:3,:])
        
        plt.imshow(fbank_feat[1:3,:])
        plt.show()
        
        plt.imshow(mfcc_feat)
        plt.show()
        
        plt.imshow(d_mfcc_feat)
        plt.show()
        
        plt.plot(sig)
        plt.show()
        break
get_feat(wavs_yes)


In [ ]:
class Signal:
    def __init__(self,rate,signal):
        self.rate = rate
        self.signal = signal
        if isinstance(self.rate, int) == False:
            raise TypeError("rate should be an interger")
        if isinstance(self.signal,  np.ndarray) == False:
            raise TypeError("signal should be a numpy array")
    
    def plot(self):
        plt.plot(self.signal)
        plt.show()
    
    def export(self, filename):
        if isinstance(filename, str) == False:
            raise TypeError("filename is not a string")
        wavfile.write(filename + '.wav', self.rate,scipy.int16(self.signal/np.linalg.norm(self.signal)*32767))




# LSTM approach 

For the rest of the project I undertake the LSTM approach which worked a lot better than the HMM approach. The HMM approach worked well for just two categories, but that isn't very conclusive since the algorithm doesn't actually have to learn anything unique. I'm writing this in restrospect and it appears that there are still two more tries on getting the algorithm neater.


In [ ]:
"""
Processing the data
"""

one_path = "C:/Users/kaweie/Documents/speech/one"
two_path = "C:/Users/kaweie/Documents/speech/two"
three_path = "C:/Users/kaweie/Documents/speech/three"
four_path = "C:/Users/kaweie/Documents/speech/four"
five_path = "C:/Users/kaweie/Documents/speech/five"
six_path = "C:/Users/kaweie/Documents/speech/six"

one_filenames = os.listdir(one_path)
two_filenames = os.listdir(two_path)
three_filenames = os.listdir(three_path)
four_filenames = os.listdir(four_path)
five_filenames = os.listdir(five_path)
six_filenames = os.listdir(six_path)

one = process_speech_data(one_path, one_filenames)
wavs_one = one.load_waves()
one.get_mfcc()

two =  process_speech_data(two_path, two_filenames)
wavs_two = two.load_waves()
two.get_mfcc()

three = process_speech_data(three_path, three_filenames)
wavs_three = three.load_waves()
three.get_mfcc()

four =  process_speech_data(four_path, four_filenames)
wavs_four = four.load_waves()
four.get_mfcc()

five = process_speech_data(five_path, five_filenames)
wavs_five = five.load_waves()
five.get_mfcc()

six =  process_speech_data(six_path, six_filenames)
wavs_six = six.load_waves()
six.get_mfcc()

mfcc_list_one = one.mfcc_list
mfcc_list_two = two.mfcc_list
mfcc_list_three = three.mfcc_list
mfcc_list_four = four.mfcc_list
mfcc_list_five = five.mfcc_list
mfcc_list_six = six.mfcc_list

mfcc_lists = [mfcc_list_one, mfcc_list_two, mfcc_list_three, mfcc_list_four, mfcc_list_five, mfcc_list_six]



In [ ]:
"""
Separating out X and Y.
"""

def get_X_Y(mfcc_lists):
    X = []
    Y = []
    for i in range(len(mfcc_lists)):
        for entry in mfcc_lists[i]:
            X.append(entry)
            
            hot_encode = np.zeros(6)
            
            hot_encode[i] = 1
            print(hot_encode)
            Y.append(hot_encode)
            
        
    return X, Y


X, Y = get_X_Y(mfcc_lists)    

The next bit of code is where we actually do the LSTM training and learning. We also try to predict the model. My predictions were not 100% accurate. I would say that I got about 80% accuracy on my test results. However, upon further inspection, some of the instances in the validation results were blank or completely not what they were labeled to be.

In [ ]:
import tflearn

learning_rate = 0.0001
training_iters = 300000

#batch = word_batch = speech_data.mfcc_batch_generator(64)
#X,Y = next(batch)

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, shuffle = True)


net = tflearn.input_data([None,26,99])
net = tflearn.lstm(net, 128, dropout = 0.8)
net = tflearn.fully_connected(net,6,activation = 'softmax')
net = tflearn.regression(net, optimizer = 'adam', learning_rate = learning_rate, loss = 'categorical_crossentropy')

model = tflearn.DNN(net, tensorboard_verbose = 0)


history = model.fit(X_train,Y_train, n_epoch = 700, validation_set=(X_test,Y_test), show_metric = True, batch_size = 32)
_y = model.predict(X)
model.save('tflearn.lstm2.model')
print(_y)

In [21]:
import pyaudio
import wave
import time

FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
CHUNK = 512
RECORD_SECONDS = 2
WAVE_OUTPUT_FILENAME = "demo.wav"
 
audio = pyaudio.PyAudio()

#Count-down from 30
print("The recording will only be 1 second.")
time.sleep(3)
print("The recording time will start once the countdown gets to zero.")
print("Speak in the given window.")
time.sleep(3)


def count_down(n):
    if n > 0:
        print(n)
        time.sleep(1)
        count_down(n-1)
    else:
        print(n)
        time.sleep(1)
        print("Speak")


count_down(10)

# start Recording
stream = audio.open(format=FORMAT, channels=CHANNELS,
                rate=RATE, input=True,
                frames_per_buffer=CHUNK)
print("recording...")
frames = []
 
for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)
print("finished recording")
 
 
#stop Recording
stream.stop_stream()
stream.close()
audio.terminate()
 
waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
waveFile.setnchannels(CHANNELS)
waveFile.setsampwidth(audio.get_sample_size(FORMAT))
waveFile.setframerate(RATE)
waveFile.writeframes(b''.join(frames))
waveFile.close()

The recording will only be 1 second.
The recording time will start once the countdown gets to zero.
Speak in the given window.
10
9
8
7
6
5
4
3
2
1
0
Speak
recording...
finished recording


In [ ]:
def prepare(filename,samplerate=16000):
    input_file = 'C:/Users/kaweie/Documents/speech/test/' + filename

    sampling_freq, sig = wavfile.read(input_file)
    
    if len(sig) < samplerate:

        sig = np.pad(sig,(samplerate - len(sig), 0), mode = 'constant')

    #elif len(sig) > samplerate:
     #   raise ValueError("There are more than {} samples!".format(str(samplerate)))

    else:
        
        sig = sig[0:samplerate]
    mfcc_feat = mfcc(sig,samplerate,nfft = 1200)
    #print("mfcc_feat shape: ", mfcc_feat.shape)
    d_mfcc_feat = delta(mfcc_feat, 2)
    #print("d_mfcc_feat shape: ", d_mfcc_feat.shape)
    fbank_feat = logfbank(sig,samplerate,nfft = 1200)
    #print("fbank_feat shape: ", fbank_feat.T.shape) 
    return (fbank_feat.T).reshape((1,26,99))











file_name = 'six3.wav'
predictions = model.predict_label(prepare(file_name))
print("The computer heard you say:")
print(predictions[0][0] + 1, "\n\n\n")
#print("If that isn't correct, the computer heard you say:")
#print(predictions[0][1] + 1, "\n\n\n")
#print(model.predict(prepare(file_name)))


# Final LSTM Algorithm

This is the final iteration of my algorithm. After much trial and error with HMMs and LSTMs and trying to generate my own recordings, I finally settled with the code below 

In [4]:
"""
In this cell I am gathering and pre-processing my data.
"""

one_path = "C:/Users/kaweie/Documents/speech/one"
two_path = "C:/Users/kaweie/Documents/speech/two"
three_path = "C:/Users/kaweie/Documents/speech/three"
four_path = "C:/Users/kaweie/Documents/speech/four"
five_path = "C:/Users/kaweie/Documents/speech/five"
six_path = "C:/Users/kaweie/Documents/speech/six"
bird_path = "C:/Users/kaweie/Documents/speech/bird"
yes_path = "C:/Users/kaweie/Documents/speech/yes"
no_path = "C:/Users/kaweie/Documents/speech/no"
cat_path = "C:/Users/kaweie/Documents/speech/cat"
dog_path = "C:/Users/kaweie/Documents/speech/dog"
tree_path = "C:/Users/kaweie/Documents/speech/tree"
bed_path = "C:/Users/kaweie/Documents/speech/bed"
wow_path = "C:/Users/kaweie/Documents/speech/wow"

one_filenames = os.listdir(one_path)
two_filenames = os.listdir(two_path)
three_filenames = os.listdir(three_path)
four_filenames = os.listdir(four_path)
five_filenames = os.listdir(five_path)
six_filenames = os.listdir(six_path)
bird_filenames = os.listdir(bird_path)
yes_filenames = os.listdir(yes_path)
no_filenames = os.listdir(no_path)
cat_filenames = os.listdir(cat_path)
dog_filenames = os.listdir(dog_path)
tree_filenames = os.listdir(tree_path)
bed_filenames = os.listdir(bed_path)
wow_filenames = os.listdir(wow_path)

one = process_speech_data(one_path, one_filenames)
wavs_one = one.load_waves()
one.get_mfcc()

two =  process_speech_data(two_path, two_filenames)
wavs_two = two.load_waves()
two.get_mfcc()

three = process_speech_data(three_path, three_filenames)
wavs_three = three.load_waves()
three.get_mfcc()

four =  process_speech_data(four_path, four_filenames)
wavs_four = four.load_waves()
four.get_mfcc()

five = process_speech_data(five_path, five_filenames)
wavs_five = five.load_waves()
five.get_mfcc()

six =  process_speech_data(six_path, six_filenames)
wavs_six = six.load_waves()
six.get_mfcc()

bird =  process_speech_data(bird_path, bird_filenames)
wavs_bird = bird.load_waves()
bird.get_mfcc()

yes =  process_speech_data(yes_path, yes_filenames)
wavs_yes = yes.load_waves()
yes.get_mfcc()

no =  process_speech_data(no_path, no_filenames)
wavs_no = no.load_waves()
no.get_mfcc()

cat =  process_speech_data(cat_path, cat_filenames)
wavs_cat = cat.load_waves()
cat.get_mfcc()

dog =  process_speech_data(dog_path, dog_filenames)
wavs_dog = dog.load_waves()
dog.get_mfcc()

tree =  process_speech_data(tree_path, tree_filenames)
wavs_tree = tree.load_waves()
tree.get_mfcc()

bed =  process_speech_data(bed_path, bed_filenames)
wavs_bed = bed.load_waves()
bed.get_mfcc()

wow =  process_speech_data(wow_path, wow_filenames)
wavs_wow = wow.load_waves()
wow.get_mfcc()

mfcc_list_one = one.mfcc_list
mfcc_list_two = two.mfcc_list
mfcc_list_three = three.mfcc_list
mfcc_list_four = four.mfcc_list
mfcc_list_five = five.mfcc_list
mfcc_list_six = six.mfcc_list
mfcc_list_bird = bird.mfcc_list
mfcc_list_yes = yes.mfcc_list
mfcc_list_no = no.mfcc_list
mfcc_list_cat = cat.mfcc_list
mfcc_list_dog = dog.mfcc_list
mfcc_list_tree = tree.mfcc_list
mfcc_list_bed = bed.mfcc_list
mfcc_list_wow = wow.mfcc_list

mfcc_lists = [mfcc_list_one, mfcc_list_two, mfcc_list_three, mfcc_list_four, mfcc_list_five, mfcc_list_six,
              mfcc_list_bird, mfcc_list_yes, mfcc_list_no, mfcc_list_cat, mfcc_list_dog, mfcc_list_tree,
              mfcc_list_bed, mfcc_list_wow]

def get_X_Y(mfcc_lists):
    X = []
    Y = []
    for i in range(len(mfcc_lists)):
        for entry in mfcc_lists[i]:
            X.append(entry)
            
            hot_encode = np.zeros(14)
            
            hot_encode[i] = 1
            Y.append(hot_encode)
            
        
    return X, Y


X, Y = get_X_Y(mfcc_lists)   



[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

...

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

This is where I actually train my algorithm

In [6]:
import tflearn

learning_rate = 0.0001
training_iters = 300000

#batch = word_batch = speech_data.mfcc_batch_generator(64)
#X,Y = next(batch)

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, shuffle = True)


net = tflearn.input_data([None,26,99])
net = tflearn.lstm(net, 128, dropout = 0.85)
net = tflearn.fully_connected(net,14,activation = 'softmax')
net = tflearn.regression(net, optimizer = 'adam', learning_rate = learning_rate, loss = 'categorical_crossentropy')

model = tflearn.DNN(net, tensorboard_verbose = 0)


history = model.fit(X_train,Y_train, n_epoch = 800, validation_set=(X_test,Y_test), show_metric = True, batch_size = 32)
_y = model.predict(X)
model.save('tflearn.lstm.more.model')
print(_y)

Training Step: 491199  | total loss: 0.39996 | time: 17.974s
| Adam | epoch: 800 | loss: 0.39996 - acc: 0.8703 -- iter: 19616/19643
Training Step: 491200  | total loss: 0.37740 | time: 21.299s
| Adam | epoch: 800 | loss: 0.37740 - acc: 0.8739 | val_loss: 0.69202 - val_acc: 0.7992 -- iter: 19643/19643
--
INFO:tensorflow:C:\Users\kaweie\Documents\speech\tflearn.lstm.more.model is not in all_model_checkpoint_paths. Manually adding it.
[[9.4580173e-01 5.6654490e-06 6.0164231e-05 ... 4.0611145e-08
  4.7972739e-02 3.0102136e-03]
 [9.3303835e-01 5.4787176e-07 7.3711167e-04 ... 4.2160337e-07
  1.5303533e-03 6.1011955e-02]
 [4.3446937e-01 1.4229772e-07 6.3319653e-06 ... 8.9707626e-08
  5.0894037e-04 5.4631859e-01]
 ...
 [4.5807984e-02 1.8790219e-05 3.6948471e-08 ... 2.7091682e-09
  4.0568183e-03 8.7660128e-01]
 [8.7178620e-03 1.5499262e-06 7.7762117e-09 ... 1.1983327e-09
  1.5762240e-04 9.2346936e-01]
 [1.4702381e-02 3.5850404e-04 2.7293868e-07 ... 6.4799474e-07
  8.7910210e-04 4.8689872e-02]]


This is where we take our trained model and make a prediction on a validation file.

In [56]:
def prepare(filename,samplerate=16000):
    input_file = 'C:/Users/kaweie/Documents/speech/test/' + filename

    sampling_freq, sig = wavfile.read(input_file)
    
    if len(sig) < samplerate:

        sig = np.pad(sig,(samplerate - len(sig), 0), mode = 'constant')

    #elif len(sig) > samplerate:
     #   raise ValueError("There are more than {} samples!".format(str(samplerate)))

    else:
        
        sig = sig[0:samplerate]
    mfcc_feat = mfcc(sig,samplerate,nfft = 1200)
    #print("mfcc_feat shape: ", mfcc_feat.shape)
    d_mfcc_feat = delta(mfcc_feat, 2)
    #print("d_mfcc_feat shape: ", d_mfcc_feat.shape)
    fbank_feat = logfbank(sig,samplerate,nfft = 1200)
    #print("fbank_feat shape: ", fbank_feat.T.shape) 
    return (fbank_feat.T).reshape((1,26,99))
file_name = 'four3.wav'
classes = ['one','two', 'three', 'four', 'five', 'six', 'bird', 'yes', 'no', 'cat', 'dog', 'tree', 'bed', 'wow']
predictions = model.predict_label(prepare(file_name))
print("The computer heard you say:")
print(classes[predictions[0][0]] , "\n\n\n")
#print("If that isn't correct, the computer heard you say:")
#print(predictions[0][1] + 1, "\n\n\n")
print(model.predict(prepare(file_name)))


The computer heard you say:
four 



[[1.1526790e-04 1.1493635e-01 1.1807749e-06 8.7342167e-01 7.4325661e-05
  4.7947628e-07 5.1915198e-05 9.8785328e-08 6.8238508e-03 1.0276661e-04
  3.8451944e-03 3.8565954e-06 2.5899773e-05 5.9711613e-04]]
